In [25]:
import os

In [26]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow/research'

In [27]:
os.chdir("../")

In [28]:
%pwd

'/Users/koushik/Desktop/MLProject-mlflow'

In [59]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [51]:
from MLProject_mlflow.constants import *
from MLProject_mlflow.utils.common import read_yaml, create_directories

In [60]:
class ConfigurationManager:
    def __init__(self):
        self.config = read_yaml(CONFIG_FILE_PATH)
        self.params = read_yaml(PARAMS_FILE_PATH)
        self.schema = read_yaml(SCHEMA_FILE_PATH)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
             root_dir=Path(config.root_dir),
             source_URL=config.source_URL,
             local_data_file=Path(config.local_data_file),
             unzip_dir=Path(config.unzip_dir)
        )

        return data_ingestion_config

In [39]:
import os
import requests
import zipfile
from MLProject_mlflow import logger
from MLProject_mlflow.utils.common import get_size

In [53]:
class DataIngestion:
    def __init__(self,config: DataIngestionConfig):
        self.config=config


    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            response = requests.get(self.config.source_URL)
            response.raise_for_status()
            with open(self.config.local_data_file, 'wb') as f:
                f.write(response.content)
            logger.info(f"Downloaded {self.config.local_data_file}")
        else:
            logger.info(f"file already exists of size: {get_size(self.config.local_data_file)}")


    def extract_zip_file(self):
        """ Extracts the contents of the zip file into the data directory
        Function returns None"""
        unzip_path=self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [61]:
try: 
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

2025-12-15 09:57:10,594: mlprojectlog: INFO: yaml file: config/config.yaml loaded successfully
2025-12-15 09:57:10,596: mlprojectlog: INFO: yaml file: params.yaml loaded successfully
2025-12-15 09:57:10,597: mlprojectlog: INFO: yaml file: schema.yaml loaded successfully
2025-12-15 09:57:10,598: mlprojectlog: INFO: created directory at path: artifacts
2025-12-15 09:57:10,599: mlprojectlog: INFO: created directory at path: artifacts/data_ingestion


TypeError: unsupported operand type(s) for /: 'PosixPath' and 'int'